In [1]:
%pip install neo4j py2neo

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import os
from tqdm import tqdm
import neo4j
from dotenv import load_dotenv

load_dotenv()


True

In [3]:
import pandas as pd 
import numpy as np
from neo4j import GraphDatabase

In [4]:
current_dir = os.getcwd()
extract_dir = os.path.join(current_dir, '..', 'Extract_Entity_Relationships')
extract_dir = os.path.abspath(extract_dir)
shields_path = os.path.join(extract_dir, 'shield_triplets.json') 
gifts_path = os.path.join(extract_dir, "gift_triplets.json")
weapon_path = os.path.join(extract_dir, "weapon_triplets.json")
spell_path = os.path.join(extract_dir, "spell_triplets.json")
with open(shields_path, "r") as f:
    shields = json.load(f)
with open(gifts_path, "r") as f:
    gifts = json.load(f)
with open(weapon_path, "r") as f:
    weapons = json.load(f)
with open(spell_path, "r") as f:
    spells = json.load(f)

In [5]:
labeled_sources = [
    (shields, "Shield"),
    (gifts, "Gift"),
    (weapons, "Weapon"),
    (spells, "Spell")
]


In [6]:
uri = "neo4j+s://f91bfb37.databases.neo4j.io"
username = "neo4j"
password = os.getenv("neo4j_password")

In [7]:
driver = GraphDatabase.driver(uri, auth=(username, password))
# Create nodes and relationships with labels
def create_typed_triplet(tx, head, relation, tail, head_label):
    cypher = f"""
    MERGE (h:{head_label} {{name: $head}})
    MERGE (t:Entity {{name: $tail}})
    MERGE (h)-[:{relation.upper()}]->(t)
    """
    tx.run(cypher, head=head, tail=tail)


In [8]:
with driver.session() as session:
    for triplet_list, label in labeled_sources:
        for head, relation, tail in triplet_list:
            session.execute_write(create_typed_triplet, head, relation, tail, label)

driver.close()